In [1]:
import os
import io
import pandas as pd
from collections import namedtuple, OrderedDict

import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util

In [2]:
def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    
    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_labels[row['class']])
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [4]:
class_labels = {'bottle': 44, 'person': 1, 'backpack': 27, 'handbag': 31, 
                'umbrella': 28, 'bowl': 51, 'cup': 47, 'clock': 85, 
                'knife': 49, 'spoon': 50, 'fork': 48, 'chair': 62, 
                'potted plant': 64, 'dining table': 67, 'keyboard': 76, 'mouse': 74, 
                'laptop': 73, 'cell phone': 77, 'scissors': 87, 'book': 84, 'monitor': 72}

In [5]:
# load csv file
filename = 'test.csv'
examples = pd.read_csv(filename)
# group same image names together
group = split(examples, 'filename')

# tfrecord path
output = 'test_test.record'
# path to images
path = os.path.join(os.getcwd(), 'images/cooc2017')

# write to tfrecord
writer = tf.python_io.TFRecordWriter(output)
for g in group:
    tf_example = create_tf_example(g, path)
    writer.write(tf_example.SerializeToString())
writer.close()
print('Successfully created the TFRecords: %s' % output)

Successfully created the TFRecords: test_test.record
